In [1]:

##Importer les données

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
import scipy.optimize as sco




In [133]:


def Load_ind_data(path = "C:/Users/Sébastien/Desktop/TP1/48_Industry_Portfolios.CSV"):

    dateparse = lambda x: pd.datetime.strptime(x,'%Y%m') 

    Data = pd.read_csv(path ,skiprows=11,nrows=1157,index_col=0,parse_dates=True,date_parser=dateparse)/100
    Data.columns = Data.columns.str.strip() #Enlever l'espace dans le titre des colonnes

    ind_names = list(Data.columns) #List of 48 industry available
    
    return {"Data": Data, "Industry_name": ind_names}

print(Load_ind_data())

Data = Load_ind_data()["Data"]

Data_select = Data[['Meals','Guns','Hshld','Oil','Txtls']]

Data_select = Data_select[Data_select.index >= '2000-01']

Data_select = Data[['Meals','Guns','Hshld','Oil','Txtls']]

Industrie_selected = Data_select.columns


{'Data':              Agric    Food    Soda    Beer   Smoke    Toys     Fun   Books  \
1926-07-01  0.0237  0.0012 -0.9999 -0.0519  0.0129  0.0865  0.0250  0.5021   
1926-08-01  0.0223  0.0268 -0.9999  0.2703  0.0650  0.1681 -0.0076  0.4298   
1926-09-01 -0.0057  0.0158 -0.9999  0.0402  0.0126  0.0833  0.0642 -0.0491   
1926-10-01 -0.0046 -0.0368 -0.9999 -0.0331  0.0106 -0.0140 -0.0509  0.0537   
1926-11-01  0.0675  0.0626 -0.9999  0.0729  0.0455  0.0000  0.0182 -0.0640   
...            ...     ...     ...     ...     ...     ...     ...     ...   
2022-07-01  0.0638  0.0368  0.0328  0.0549  0.0056  0.0563  0.1704  0.1208   
2022-08-01  0.0523 -0.0046 -0.0440 -0.0187 -0.0012 -0.0577 -0.0226 -0.0500   
2022-09-01 -0.0703 -0.0765 -0.0814 -0.0521 -0.1055 -0.1543 -0.0410 -0.1338   
2022-10-01  0.1379  0.1112  0.0713  0.0949  0.1208  0.0548  0.1498  0.0882   
2022-11-01  0.0253  0.0284  0.0769  0.0323  0.0557  0.0979  0.0724  0.1570   

             Hshld   Clths  ...   Boxes   Trans   Whls

C:\Users\Sébastien\AppData\Local\Temp\ipykernel_8228\3224556917.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda x: pd.datetime.strptime(x,'%Y%m')


In [42]:

## Without risk-free asset :

# Determination of the maximum return portfolio (under certain constraints) :

#Initial weights irrevelant since we can guess anything

def Max_return_ptf(E_return, bounds = (0, 1)):
      
    #Initialize variables
    
    Initial_weights = np.array([1 / len(E_return)] * len(E_return))
    bounds = tuple(bounds for w in Initial_weights)
    Industrie_selected = E_return.index
    constraints_max_ret = ( {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # Sum of the weigth need to sum to 1
    
# Define objective fonction to minimize :
    def Max_return_obj (weigth, E_return = E_return.values):
        return - np.dot(weigth.T,E_return)
# Compute the maximum return portfolio by optimisation under constraints: 

    Max_ret_PTF = sco.minimize(

      fun = Max_return_obj, 
      x0 = Initial_weights, 
      method = 'SLSQP',
      bounds = bounds, 
      constraints = constraints_max_ret
    
    )

    Max_ret_weigth = pd.DataFrame(Max_ret_PTF['x'], index=Industrie_selected,columns = ['Weigth'])

    Max_ret_return = pd.DataFrame([np.dot(Max_ret_PTF['x'].T, E_return.values)], columns = ['return'])
    
    Max_return = {'Max_ret_return' : Max_ret_return,
                       'Max_ret_weigth' : Max_ret_weigth.T}
    return(Max_return)
  




In [26]:
E_return = Data_select.mean()
Max_return_result = Max_return_ptf(E_return, bounds= (-2,2))
Max_return_result
  

({'Max_ret_return':      return
  0  0.030569,
  'Max_ret_weigth':         Meals  Guns  Hshld  Oil  Txtls
  Weigth    2.0   2.0   -2.0  1.0   -2.0},
 ((-2, 2), (-2, 2), (-2, 2), (-2, 2), (-2, 2)))

In [39]:

##### Determination of the minimum-variance  portfolio  :

def Min_var_ptf(E_return, E_cov, bounds = (-2,2)):
    
    #Initialize variables
    
    Initial_weights = np.array([1 / len(E_return)] * len(E_return))
    bounds = tuple(bounds for w in Initial_weights)
    Industrie_selected = E_return.index
    constraints_min_var = ( {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # Sum of the weigth need to sum to 1
   
    
    # Define objective fonction to minimize :

    def Min_Variance_obj (weigth,Cov_matrix = E_cov) : 
        return (0.5*np.dot(np.dot(weigth.T,Cov_matrix),weigth))
# Compute the minimum variance portfolio by optimisation under constraints: 

    Min_var_PTF = sco.minimize(

      fun = Min_Variance_obj, 
      x0 = Initial_weights, 
      method = 'SLSQP',
      constraints = constraints_min_var,
      bounds = bounds
    
    )

    Min_var_weigth = pd.DataFrame(Min_var_PTF['x'], index=Industrie_selected, columns= ['Weigth'])

    Min_var_return = pd.DataFrame([np.dot(Min_var_PTF['x'].T, E_return.values)], columns= ['return'])
    
    Min_var = {'Min_var_return': Min_var_return,
                     'Min_var_weigth': Min_var_weigth.T}
  
    return(Min_var)



In [86]:
E_return = Data_select.mean()
E_cov = Data_select.cov()
Min_var_result = Min_var_ptf(E_return, E_cov, bounds = (-2,2))
Nbr_PTF = 100

np.zeros((len(E_return), Nbr_PTF))

E_return



Meals    0.010559
Guns     0.014635
Hshld    0.006298
Oil      0.009881
Txtls    0.008552
dtype: float64

In [91]:
##### Determination of the optimal  portfolio  for a given return  :


def Ptf_target_optimization(E_return, E_cov, Nbr_PTF, bounds = (-2, 2)):
    
    #Define Variables
    
    Min_var_result = Min_var_ptf(E_return, E_cov, bounds)
    Max_return_result = Max_return_ptf(E_return, bounds)
    
    Initial_weights = np.array([1 / len(E_return)] * len(E_return))
    bounds = tuple(bounds for w in Initial_weights)
    
    
    
    Target_return = np.linspace (Min_var_result['Min_var_return'].iloc[0]['return'], Max_return_result['Max_ret_return'].iloc[0]['return'], Nbr_PTF)
    Weigth_names = E_return.index
    Weigth_level = np.zeros((len(E_return), Nbr_PTF)) #Initialize the array
    Var_level = np.zeros((1, Nbr_PTF))
    
    
    # Define objective fonction to minimize :

    def Min_Variance_obj (weigth, Cov_matrix = E_cov) : 
        return (0.5*np.dot(np.dot(weigth.T, Cov_matrix), weigth))
    
# Optimization for each return target :

    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x*E_return.values) - target}, # Optimization for a target return 
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    for i in range(0,Nbr_PTF):
        target = Target_return[i]
        Efficient_frontier_result =  sco.minimize(
            fun = Min_Variance_obj, 
            x0 = Initial_weights, 
            method = 'SLSQP', 
            constraints = constraints,
            bounds = bounds
        ) 
        
        Weigth_level[:,i] = Efficient_frontier_result ['x']
        Var_level[0,i] = np.sqrt(2*Efficient_frontier_result['fun'])
    
    Return_effectiv = np.dot(E_return.values, Weigth_level)
    Efficient_frontiere = np.concatenate((np.array([Target_return]), Var_level))
    
    # Data formatting 
    
    Efficient_frontiere_df = pd.DataFrame(Efficient_frontiere.T, columns=['Returns','Standard_deviation'])
    Efficient_frontiere_df.set_index('Standard_deviation', inplace=True)
    
    Efficient_frontiere_weigth = pd.DataFrame(Weigth_level.T, columns=Weigth_names)
    
    Efficient_frontiere_result = {'Efficient_frontiere': Efficient_frontiere_df,
                                           'Efficient_frontiere_weigth': Efficient_frontiere_weigth}
    
    return(Efficient_frontiere_result)






In [134]:
Ptf_target_optimization(E_return, E_cov, Nbr_PTF = 100, bounds = (-2, 2))

{'Efficient_frontiere':                      Returns
 Standard_deviation          
 0.037757            0.009991
 0.038101            0.010199
 0.047892            0.010407
 0.038926            0.010615
 0.039403            0.010823
 ...                      ...
 0.155148            0.029738
 0.162573            0.029946
 0.170825            0.030154
 0.179788            0.030362
 0.189363            0.030569
 
 [100 rows x 1 columns],
 'Efficient_frontiere_weigth':        Meals      Guns     Hshld       Oil     Txtls
 0   0.294806  0.281711  0.465686  0.138097 -0.180300
 1   0.300912  0.305068  0.440730  0.136044 -0.182755
 2   0.206791  0.252461  0.159622  0.198606  0.182521
 3   0.313032  0.351715  0.390789  0.132332 -0.187868
 4   0.319144  0.375047  0.365870  0.130447 -0.190508
 ..       ...       ...       ...       ...       ...
 95  2.000000  2.000000 -2.000000  0.374253 -1.374253
 96  2.000000  2.000000 -2.000000  0.530690 -1.530690
 97  2.000000  2.000000 -2.000000  0.687126 

In [121]:

## With risk-free asset :

def Ptf_target_optimization_W_Rf(E_return, E_cov, Expected_Risk_free, Nbr_PTF, bounds = (-2,2)):
    
     #Define Variables
    
    Max_return_result = Max_return_ptf(E_return, bounds)
    
    Initial_weights = np.array([1 / len(E_return)] * len(E_return))
    bounds = tuple(bounds for w in Initial_weights)
    
    
    Target_return = np.linspace(Expected_Risk_free, Max_return_result['Max_ret_return'].iloc[0]['return'], Nbr_PTF)
    Weigth_names =  np.concatenate((np.array(['Risk_free']) , E_return.index.values))
    
    Var_level_W_O_S = np.zeros((1, Nbr_PTF))
    Weigth_level_W_O_S = np.zeros((len(E_return) + 1 , Nbr_PTF)) #Initialize the array
    
    
    
    # Define objective fonction to minimize :

    def Min_Variance_obj (weigth, Cov_matrix = E_cov) : 
        return (0.5*np.dot(np.dot(weigth.T, Cov_matrix), weigth))
# Constraint :

    constraints_W_RF = (
          {'type': 'eq', 'fun': lambda x: np.sum(x*(E_return.values-Expected_Risk_free)) - (target-Expected_Risk_free)}) # Optimization for a target return 


    for i in range(0,Nbr_PTF):
        target = Target_return[i]
        Efficient_frontier_r =  sco.minimize(
            fun = Min_Variance_obj, 
            x0 = Initial_weights, 
            method = 'SLSQP', 
            constraints = constraints_W_RF,
            bounds = bounds
        ) 
        
        Var_level_W_O_S[0,i] = np.sqrt(2*Efficient_frontier_r['fun'])
        Weigth_level_W_O_S[1:6,i] = Efficient_frontier_r['x']
        Weigth_level_W_O_S[0,i] = 1-np.sum(Efficient_frontier_r['x'])
    
    Expected_return_W_RF= np.concatenate((np.array([Expected_Risk_free]),E_return))
    Return_effectiv= np.dot(Expected_return_W_RF,Weigth_level_W_O_S)
    
    Efficient_frontiere = np.concatenate((np.array([Return_effectiv]),Var_level_W_O_S))
    
    # Data formatting 
    
    Efficient_frontiere_df = pd.DataFrame(Efficient_frontiere.T,columns=['Returns','Standard_deviation'])
    Efficient_frontiere_df.set_index('Standard_deviation', inplace=True)
    Efficient_frontiere_weigth = pd.DataFrame(Weigth_level_W_O_S.T,columns=Weigth_names)
    
    Efficient_frontiere_result = dict([('Efficient_frontiere',Efficient_frontiere_df),
                                           ('Efficient_frontiere_weigth',Efficient_frontiere_weigth)])
    
    return(Efficient_frontiere_result)



##Créer une fonction plot ()

In [131]:
# Input : 

# Importation of the SOFR :




def Load_rf(path = "C:/Users/Sébastien/Desktop/TP1/TP1/SOFR30DAYAVG.xls", date = '2021-08-01'):

    Data_SOFR = pd.read_excel(path)

    Data_SOFR['observation_date'] = pd.to_datetime(Data_SOFR['observation_date'], format='%Y-%m-%d')
    Data_SOFR.set_index('observation_date', inplace=True)

    Data_SOFR=Data_SOFR.dropna()



    Expected_Risk_free = Data_SOFR[Data_SOFR.index >= date].mean() # do again 
    
    return  Expected_Risk_free.iloc[0]

Load_rf()

1.149232404371585

In [125]:
Ptf_target_optimization_W_Rf(E_return, E_cov, Expected_Risk_free, Nbr_PTF, bounds = (-2,2))






WindowsPath('c:/Users/Sébastien/Desktop/TP1/TP1/SOFR30DAYAVG.xls')